In [65]:
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup

In [11]:
for i in range(1, 10):
    print(i)
    time.sleep(3)

1
2
3
4
5
6
7
8
9


In [3]:
import getpass

In [5]:
X = input('password please')

password please Hola!!


In [7]:
X

'Hola!!'

In [9]:
client_id = getpass.getpass()       #alphanumeric string provided under "personal use script"
client_secret = getpass.getpass()   #alphanumeric string provided as "secret"
user_agent = getpass.getpass()      #the name of your application
username = getpass.getpass()        #your reddit username
password =  getpass.getpass()       #your reddit password

 ········
 ········
 ········
 ········
 ········


Now we're on our way to retrieving our access token; we'll use the basic authentication framework to get there.

In [21]:
auth = requests.auth.HTTPBasicAuth(client_id, client_secret)

data = {
    'grant_type': 'password',
    'username': username,
    'password': password
}

In [23]:
#create an informative header for your application
headers = {'User-Agent': 'dsb0826project3/0.0.1'}

res = requests.post(
    'https://www.reddit.com/api/v1/access_token',
    auth=auth,
    data=data,
    headers=headers)

print(res)

<Response [200]>


Hopefully upon running the above, you received a successful response code and can save your token. These should last for about two hours by default.

In [29]:
#retrieve access token
token = res.json()['access_token']

Now let's add your access token to the headers and verify that you can successfully submit a call to the api.

In [31]:
headers['Authorization'] = f'bearer {token}'

requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).status_code == 200

True

If all went correctly, we can finally create a simple request.

In [33]:
base_url = 'https://oauth.reddit.com/r/'
subreddit = 'travel'

res = requests.get(base_url+subreddit, headers=headers)

Explore the response object. Where is our submission data? How many posts were retrieved by default?

In [118]:
#check out response object
res.json()

{'kind': 'Listing',
 'data': {'after': 't3_1fvhvbe',
  'dist': 26,
  'modhash': None,
  'geo_filter': None,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'travel',
     'selftext': '**READ THE NEW LAYOVER FAQ:** [**https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers**](https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers)\n\nAll layover questions will be removed unless your situation is unique and cannot be answered by the wiki.\n\n**Members of the community**: please report any layover questions that can be answered by the wiki and we will remove them promptly.\n\nSelf-transfers times are not covered under this new guideline and wiki.',
     'author_fullname': 't2_3nmwc',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'All Layover Questions - READ THIS NOTICE',
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/travel',
     'hidden': False,
     'pwls': 6,
  

In [120]:
res.json().keys()

dict_keys(['kind', 'data'])

In [124]:
res.json()['data']

{'after': 't3_1fvhvbe',
 'dist': 26,
 'modhash': None,
 'geo_filter': None,
 'children': [{'kind': 't3',
   'data': {'approved_at_utc': None,
    'subreddit': 'travel',
    'selftext': '**READ THE NEW LAYOVER FAQ:** [**https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers**](https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers)\n\nAll layover questions will be removed unless your situation is unique and cannot be answered by the wiki.\n\n**Members of the community**: please report any layover questions that can be answered by the wiki and we will remove them promptly.\n\nSelf-transfers times are not covered under this new guideline and wiki.',
    'author_fullname': 't2_3nmwc',
    'saved': False,
    'mod_reason_title': None,
    'gilded': 0,
    'clicked': False,
    'title': 'All Layover Questions - READ THIS NOTICE',
    'link_flair_richtext': [],
    'subreddit_name_prefixed': 'r/travel',
    'hidden': False,
    'pwls': 6,
    'link_flair_css_class': 'modpost',
    'down

In [126]:
res.json()['data'].keys()

dict_keys(['after', 'dist', 'modhash', 'geo_filter', 'children', 'before'])

In [196]:
print(res.json()['data']['after'])
print(res.json()['data']['before'])

t3_1fvhvbe
None


In [144]:
print(type(res.json()['data']['children']))
print(len(res.json()['data']['children']))
res.json()['data']['children']

<class 'list'>
26


[{'kind': 't3',
  'data': {'approved_at_utc': None,
   'subreddit': 'travel',
   'selftext': '**READ THE NEW LAYOVER FAQ:** [**https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers**](https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers)\n\nAll layover questions will be removed unless your situation is unique and cannot be answered by the wiki.\n\n**Members of the community**: please report any layover questions that can be answered by the wiki and we will remove them promptly.\n\nSelf-transfers times are not covered under this new guideline and wiki.',
   'author_fullname': 't2_3nmwc',
   'saved': False,
   'mod_reason_title': None,
   'gilded': 0,
   'clicked': False,
   'title': 'All Layover Questions - READ THIS NOTICE',
   'link_flair_richtext': [],
   'subreddit_name_prefixed': 'r/travel',
   'hidden': False,
   'pwls': 6,
   'link_flair_css_class': 'modpost',
   'downs': 0,
   'thumbnail_height': None,
   'top_awarded_type': None,
   'hide_score': False,
   'name': 't3_

In [223]:
#for item in res.json()['data']['children']:
    #print(f"Title: {item['data']['title']}, {item['data']['name']}")
    #print('\n')
    #print(f"Post: {item['data']['selftext']}")
    #print('='*150)
    #print('='*150)

In [202]:
first_batch_of_26 = res.json()['data']['children']

In [208]:
data = []

for item in first_batch_of_26:
    record = {
        'post_id' : item['data']['name'],
        'post_title' : item['data']['title'],
        'post_text' : item['data']['selftext']
    }
    data.append(record)

In [214]:
reddit_travel = pd.DataFrame(data)
reddit_travel.shape

(26, 3)

In [216]:
reddit_travel.head()

,post_id,post_title,post_text
0,t3_1dzc3zh,All Layover Questions - READ THIS NOTICE,**READ THE NEW LAYOVER FAQ:** [**https://www.r...
1,t3_1fv4vli,Just how many people have been scammed by edre...,I recently found I was charged 60 euros by a t...
2,t3_1fv4z6a,Required to carry significant amount of cash t...,I was told by my tour organizer that we are re...
3,t3_1fvgocr,First time in a hostel,I am heading out to London and Wales next week...
4,t3_1fvaxy8,What is it with every business begging for Goo...,"I’m currently in Egypt (spoiler: I love it), a..."


In [1]:
#reddit_travel.to_csv('reddit_travel.csv', index = False)

Let's now make use of the fact that we can pass a parameters dictionary to increase the size of our request then create a dataframe of our submissions.

In [227]:
#modify request
params = {
    'limit': 100,
    'after': 't3_1fvhvbe'
}

res = requests.get(base_url+subreddit,
                   headers=headers,
                  params=params)

In [229]:
#check status code
res.status_code

200

In [233]:
next_batch_of_100 = res.json()['data']['children']

In [235]:
data = []

for item in next_batch_of_100:
    record = {
        'post_id' : item['data']['name'],
        'post_title' : item['data']['title'],
        'post_text' : item['data']['selftext']
    }
    data.append(record)

In [237]:
len(data)

100

In [239]:
reddit_travel = pd.DataFrame(data)
reddit_travel.shape

(100, 3)

In [243]:
reddit_travel.tail()

,post_id,post_title,post_text
95,t3_1fukpac,Dolomites in November,"Hi guys, traveled a lot through Europe mostly ..."
96,t3_1fukfwi,Fiji airlines - baggage allowance,Looking to fly to Fiji in October 2025. Flying...
97,t3_1fukf55,5 days left between Berlin and returning to Am...,"Basically, I booked a return ticket (to Toront..."
98,t3_1fujznz,Experience with TSA and garment bags,I am attending my sons wedding and would like ...
99,t3_1fujtnq,Rabbie's Tours versus self-guided Highlands Ro...,"My husband and I (both 30, from the US) are pl..."


In [249]:
output_path = 'reddit_travel.csv'

In [253]:
# append new data to existing file
reddit_travel.to_csv(output_path, mode = 'a', header = False, index = False)

In [ ]:
#create a dataframe of your submissions


**Exercise**: write a loop to retrieve the 1000 most recent submissions. What parameters of the submissions endpoint will be most helpful for you here? [To the docs!](https://www.reddit.com/dev/api/)